<a href="https://colab.research.google.com/github/IM07813/Transformers/blob/main/essay_eval__.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers




Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 44.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 33.4 MB/s eta 0:00:00


In [2]:
 !pip install --upgrade tensorflow transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
# Import libraries
import tensorflow as tf
import transformers
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer

# Load pre-trained model and tokenizer
model_name = "bert-base-cased"
model = TFAutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5) # Assuming 6-point scale for essay scores
tokenizer = AutoTokenizer.from_pretrained(model_name)
print(transformers.__version__)
print(tf.__version__)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


4.29.2
2.12.0


In [4]:
# Preprocess essay data
texts = ["This is an example essay.", "This is another example essay."] # Replace with your essay texts
labels = [4, 5] # Replace with your essay labels
max_length = 512 # Adjust as needed

In [5]:
input_ids = []
attention_masks = []

for text in texts:
  encoding = tokenizer.encode_plus(
    text,
    max_length=max_length,
    truncation=True,
    padding="max_length",
    add_special_tokens=True,
    return_tensors="tf"
  )
  input_ids.append(encoding["input_ids"])
  attention_masks.append(encoding["attention_mask"])

input_ids = tf.concat(input_ids, axis=0)
attention_masks = tf.concat(attention_masks, axis=0)
labels = tf.convert_to_tensor(labels)

In [6]:
# Create TensorFlow dataset
batch_size = 8 # Adjust as needed
dataset = tf.data.Dataset.from_tensor_slices(({"input_ids": input_ids, "attention_mask": attention_masks}, labels))
dataset = dataset.shuffle(len(input_ids)).batch(batch_size)

In [7]:
# Fine-tune model on essay data
model.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5),
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=["accuracy"]
)

In [8]:
tokenizer.vocab_size

28996

In [9]:
len(tokenizer.get_vocab())

28996

In [10]:
tf.shape(batch_size)

<tf.Tensor: shape=(0,), dtype=int32, numpy=array([], dtype=int32)>

In [11]:
tf.shape(attention_masks)

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([  2, 512], dtype=int32)>

In [12]:
model.fit(dataset, epochs=3)  # Adjust as needed
    



Epoch 1/3
1/1 [==============================] - 63s 63s/step - loss: nan - accuracy: 0.5000
Epoch 2/3
1/1 [==============================] - 0s 326ms/step - loss: nan - accuracy: 0.0000e+00
Epoch 3/3
1/1 [==============================] - 0s 236ms/step - loss: nan - accuracy: 0.0000e+00
